In [2]:
import re
import pandas as pd
from datetime import datetime
from main import parse_current_state_log,parse_order_log, extract_position_events, extract_execution_details

df_log = parse_current_state_log("current_state.log")
print(df_log.head())

df_events = extract_position_events(df_log)
print("Extracted events dataframe:")
print(df_events)
print("=" * 60)


                   timestamp  \
0 2025-01-01 07:31:46.294200   
1 2025-01-01 18:25:49.234900   
2 2025-01-01 20:43:47.320100   
3 2025-01-02 00:29:42.565200   
4 2025-01-03 16:01:45.906600   

                                           positions  
0  {'WUSDT': -1, 'LINKUSDT': 1, 'RUNEUSDT': 1, 'S...  
1  {'WUSDT': -1, 'LINKUSDT': 1, 'RUNEUSDT': 1, 'S...  
2  {'WUSDT': -1, 'RUNEUSDT': 1, 'STORJUSDT': -1, ...  
3  {'WUSDT': -1, 'RUNEUSDT': 1, 'STORJUSDT': -1, ...  
4  {'WUSDT': -1, 'RUNEUSDT': 1, 'POLUSDT': 1, 'ZR...  
Extracted events dataframe:
        symbol                  timestamp action  old_position  new_position
0      ZROUSDT 2025-01-01 07:31:46.294200   open             0             1
1     MASKUSDT 2025-01-01 07:31:46.294200   open             0            -1
2    STORJUSDT 2025-01-01 07:31:46.294200   open             0            -1
3      BNBUSDT 2025-01-01 07:31:46.294200   open             0            -1
4     RUNEUSDT 2025-01-01 07:31:46.294200   open             0  

## Return a list with all traded pairs

this list passed into the binance_feed.py program returns ohlcv data for all required pairs


In [3]:
df_order_log = parse_order_log("binance_bot_data/order_new.log")
print("Order log dataframe:")
print(df_order_log.head())
print("=" * 60)

df_executions = extract_execution_details(df_order_log)
print("Execution details dataframe:")
print(df_executions)
print("=" * 60)
  
pairs = set()

for pair in df_events["symbol"]:
    pairs.add(pair)

print(list(pairs))

Order log dataframe:
                timestamp                                            message
0 2025-03-06 15:49:28.883  INFO;broker_web_binancefut_mel_cm2;splitting o...
1 2025-03-06 15:49:29.404  INFO;broker_web_binancefut_mel_cm2;Preparing o...
2 2025-03-06 15:49:29.404  INFO;broker_web_binancefut_mel_cm2;Converted 6...
3 2025-03-06 15:49:29.405  INFO;broker_web_binancefut_mel_cm2;Received mi...
4 2025-03-06 15:49:29.405  INFO;broker_web_binancefut_mel_cm2;sending ord...
Execution details dataframe:
                  timestamp        pair  execution_price
0   2025-03-06 15:50:32.716    FLOWUSDT          0.44600
1   2025-03-06 15:50:32.716  RENDERUSDT          4.07500
2   2025-03-06 15:52:05.575   MANTAUSDT          0.30650
3   2025-03-06 15:52:05.575     CHZUSDT          0.05028
4   2025-03-06 15:54:08.845     POLUSDT          0.25381
..                      ...         ...              ...
107 2025-03-10 04:15:28.523    MINAUSDT          0.25890
108 2025-03-10 07:47:04.207    N

In [ ]:
traded_pairs_list = ['CELOUSDT', 'MKRUSDT', 'PYTHUSDT', 'LINKUSDT', 'SEIUSDT', 'SANDUSDT', 'CHZUSDT', 'INJUSDT', 'IDUSDT', 'MEWUSDT', 'XLMUSDT', 'RENDERUSDT', 'TAOUSDT', 'AXSUSDT', 'MASKUSDT', 'BCHUSDT', 'NEOUSDT', 'TONUSDT', 'GASUSDT', 'ATOMUSDT', 'THETAUSDT', 'CFXUSDT', 'APEUSDT', 'EOSUSDT', 'ADAUSDT', 'IMXUSDT', 'ZROUSDT', 'XTZUSDT', 'ZILUSDT', 'BLURUSDT', 'ORDIUSDT', 'STORJUSDT', 'ZRXUSDT', 'BOMEUSDT', 'POLUSDT', 'KAIAUSDT', 'ZETAUSDT', 'EGLDUSDT', 'FETUSDT', 'CKBUSDT', 'APTUSDT', 'BTCUSDT', 'GRTUSDT', 'WUSDT', 'GLMUSDT', 'STXUSDT', 'SAFEUSDT', 'FILUSDT', 'OPUSDT', 'RUNEUSDT', 'SNXUSDT', 'KSMUSDT', 'DOTUSDT', 'ETHUSDT', 'AVAXUSDT', 'UNIUSDT', 'COMPUSDT', 'AAVEUSDT', 'ENSUSDT', 'DOGEUSDT', 'AEVOUSDT', 'GMTUSDT', 'FLOWUSDT', 'MINAUSDT', 'ZKUSDT', 'STRKUSDT', 'ARBUSDT', 'ETCUSDT', 'ROSEUSDT', 'MEMEUSDT', 'YFIUSDT', 'SOLUSDT', 'POLYXUSDT', 'NOTUSDT', 'PIXELUSDT', 'BATUSDT', 'ONTUSDT', 'BNBUSDT', 'MANTAUSDT', '1INCHUSDT', 'LPTUSDT', 'NEARUSDT', 'LTCUSDT', 'ZECUSDT', 'ICPUSDT', 'MANAUSDT']
len(traded_pairs_list)

86

## extract clusters

In [4]:
import os
import json

def get_all_coins(directory):
    all_coins = set()
    # Iterate over all JSON files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as file:
                data = json.load(file)
            # Extract coins from the "peers" field
            peers_dict = data.get("peers", {})
            for symbol, peers in peers_dict.items():
                all_coins.update(peers)
            # Extract coins from the "selected" field
            selected_pairs = data.get("selected", [])
            for pair in selected_pairs:
                # Split the pair string by '__' to extract individual coins
                coins = pair.split("__")
                all_coins.update(coins)
    return all_coins

# Example usage: specify your folder path
folder_path = "binance_clusters"  # Replace with your folder path
unique_coins = get_all_coins(folder_path)
unique_coins_list = list(unique_coins)
print("Unique coins found:", unique_coins_list)





Unique coins found: ['LINKUSDT', 'LTCUSDT', 'NEOUSDT', 'FETUSDT', 'BLURUSDT', 'ORDIUSDT', 'MANTAUSDT', 'CKBUSDT', 'IDUSDT', 'SANDUSDT', 'BCHUSDT', 'IMXUSDT', 'DOGEUSDT', 'DRIFTUSDT', 'BTCUSDT', 'MKRUSDT', 'POLUSDT', 'STRKUSDT', 'DYDXUSDT', 'OPUSDT', 'APTUSDT', 'ZRXUSDT', 'MASKUSDT', '1INCHUSDT', 'MINAUSDT', 'GMTUSDT', 'STORJUSDT', 'THETAUSDT', 'ROSEUSDT', 'ZKUSDT', 'XRPUSDT', 'ZROUSDT', 'SNXUSDT', 'ADAUSDT', 'SUIUSDT', 'WUSDT', 'NOTUSDT', 'FLOWUSDT', 'CELOUSDT', 'ENSUSDT', 'FILUSDT', 'POLYXUSDT', 'SOLUSDT', 'BNBUSDT', 'CFXUSDT', 'MEWUSDT', 'BATUSDT', 'KAVAUSDT', 'TONUSDT', 'ETHUSDT', 'AEVOUSDT', 'ZETAUSDT', 'COMPUSDT', 'CAKEUSDT', 'INJUSDT', 'NEARUSDT', 'CHZUSDT', 'ZILUSDT', 'AXSUSDT', 'RENDERUSDT', 'ZECUSDT', 'STXUSDT', 'AAVEUSDT', 'SAFEUSDT', 'PYTHUSDT', 'DOTUSDT', 'GASUSDT', 'EOSUSDT', 'ICPUSDT', 'APEUSDT', 'MANAUSDT', 'SEIUSDT', 'VETUSDT', 'ATOMUSDT', 'GRTUSDT', 'XTZUSDT', 'KAIAUSDT', 'KSMUSDT', 'UNIUSDT', 'GLMUSDT', 'YFIUSDT', 'LPTUSDT', 'PENDLEUSDT', 'QNTUSDT', 'MEMEUSDT', 'TIAUS

In [12]:


# Create a union by converting both lists to sets and taking the union
combined_set = set(unique_coins_list).union(traded_pairs_list)

# Convert the result back to a list if needed
combined_list = list(combined_set)

print("Combined list: ")
print(combined_list)

Combined list: 
['LINKUSDT', 'BNBUSDT', 'CFXUSDT', 'MEWUSDT', 'BATUSDT', 'XLMUSDT', 'KAVAUSDT', 'LTCUSDT', 'NEOUSDT', 'FETUSDT', 'BLURUSDT', 'TONUSDT', 'ETHUSDT', 'ORDIUSDT', 'MANTAUSDT', 'CKBUSDT', 'AEVOUSDT', 'ZETAUSDT', 'COMPUSDT', 'IDUSDT', 'SANDUSDT', 'CAKEUSDT', 'BCHUSDT', 'INJUSDT', 'NEARUSDT', 'CHZUSDT', 'IMXUSDT', 'DRIFTUSDT', 'DOGEUSDT', 'BTCUSDT', 'ZILUSDT', 'AXSUSDT', 'MKRUSDT', 'RENDERUSDT', 'POLUSDT', 'ZECUSDT', 'STRKUSDT', 'STXUSDT', 'DYDXUSDT', 'AAVEUSDT', 'SAFEUSDT', 'OPUSDT', 'APTUSDT', 'PYTHUSDT', 'ZRXUSDT', 'MASKUSDT', '1INCHUSDT', 'DOTUSDT', 'GASUSDT', 'EOSUSDT', 'ICPUSDT', 'APEUSDT', 'TAOUSDT', 'MANAUSDT', 'SEIUSDT', 'MINAUSDT', 'VETUSDT', 'ATOMUSDT', 'GMTUSDT', 'STORJUSDT', 'THETAUSDT', 'ROSEUSDT', 'GRTUSDT', 'ZKUSDT', 'XTZUSDT', 'KAIAUSDT', 'KSMUSDT', 'UNIUSDT', 'XRPUSDT', 'ZROUSDT', 'SNXUSDT', 'GLMUSDT', 'YFIUSDT', 'LPTUSDT', 'BOMEUSDT', 'PENDLEUSDT', 'RUNEUSDT', 'QNTUSDT', 'MEMEUSDT', 'ADAUSDT', 'SUIUSDT', 'WUSDT', 'TIAUSDT', 'ETCUSDT', 'AXLUSDT', 'ONTUSDT', '

In [3]:

# Find coins in list1 that are not in list2
coins_only_in_traded_pairs_list = set(traded_pairs_list) - unique_coins

# Alternatively, if you want coins in list2 not in list1:
coins_only_in_unique_coins_list = unique_coins - set(traded_pairs_list)

print("Coins traded but not in clusters:")
print(coins_only_in_traded_pairs_list)

print("Coins in clusters but not traded:")
print(coins_only_in_unique_coins_list)

Coins trades but not in clusters:
{'BOMEUSDT', 'RUNEUSDT', 'XLMUSDT', 'TAOUSDT', 'PIXELUSDT'}
Coins in clusters but not traded:
{'PENDLEUSDT', 'CAKEUSDT', 'KAVAUSDT', 'VETUSDT', 'QNTUSDT', 'SUIUSDT', 'TIAUSDT', 'AXLUSDT', 'DRIFTUSDT', 'XRPUSDT', 'DYDXUSDT'}
